# Multi-Cohort Running and Turning Behavior Analysis

This notebook analyzes multiple cohort behavioral analysis CSV files (merged together) and generates summary plots showing:
- Running velocity (cm/s) with average and SEM per experiment day
- Total run distance per experiment day **normalized by session duration (m/minute)**
- Time spent running (percentage) per experiment day
- Turning velocity (deg/s) with average and SEM per experiment day
- Total turn distance per experiment day **normalized by session duration (deg/minute)**
- Time spent turning (percentage) per experiment day

Individual mouse values are shown with consistent colors across days and cohorts. Analysis is performed across all cohorts grouped by experiment day type.

**Note:** Distance metrics are normalized by total recording time to account for differences in session duration across experiment days. This ensures fair comparison between training days and experiment days with different recording lengths.


In [19]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from typing import Dict, List, Optional
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 15
plt.rcParams['font.family'] = 'Arial'
plt.rcParams['font.sans-serif'] = ['Arial', 'DejaVu Sans']


In [10]:
# Configuration
#----------------------------
# Paths to cohort CSV files (can specify multiple cohorts)
# CSV files should be located 2 levels above _processedData folder
cohort_csv_paths = [
    Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20241125_Cohort1_rotation/cohort_behavioral_analysis.csv').expanduser(),
    Path('/home/ikharitonov/RANCZLAB-NAS/data/ONIX/20250409_Cohort3_rotation/cohort_behavioral_analysis.csv').expanduser(),
    # Add more cohort CSV paths here as needed
    # Path('/path/to/cohort2/cohort_behavioral_analysis.csv').expanduser(),
]

# Output directory for plots (use first cohort's parent directory or specify a common location)
if len(cohort_csv_paths) > 0:
    # Use parent directory of first cohort
    output_dir = cohort_csv_paths[0].parent
else:
    output_dir = Path('.')

print(f"Number of cohort CSV files: {len(cohort_csv_paths)}")
for i, path in enumerate(cohort_csv_paths, 1):
    print(f"  Cohort {i}: {path}")
print(f"\nOutput directory: {output_dir}")


Number of cohort CSV files: 2
  Cohort 1: /home/ikharitonov/RANCZLAB-NAS/data/ONIX/20241125_Cohort1_rotation/cohort_behavioral_analysis.csv
  Cohort 2: /home/ikharitonov/RANCZLAB-NAS/data/ONIX/20250409_Cohort3_rotation/cohort_behavioral_analysis.csv

Output directory: /home/ikharitonov/RANCZLAB-NAS/data/ONIX/20241125_Cohort1_rotation


In [11]:
# Load and merge multiple cohort CSV files
#----------------------------
dfs = []
cohort_names = []

for i, csv_path in enumerate(cohort_csv_paths, 1):
    if not csv_path.exists():
        print(f"⚠️ Warning: Cohort CSV not found at {csv_path}, skipping...")
        continue
    
    # Load CSV
    df_temp = pd.read_csv(csv_path)
    
    # Extract cohort directory name from path (e.g., "20250409_Cohort3_rotation" or "Cohort1_rotation")
    # Use the parent directory name as the cohort identifier
    cohort_name = csv_path.parent.name
    
    # If parent is just the filename or empty, try grandparent
    if not cohort_name or cohort_name == csv_path.parent:
        cohort_name = csv_path.parent.parent.name
    
    # If still not found, use a default name
    if not cohort_name or cohort_name == '.':
        cohort_name = f"Cohort{i}"
    
    # Add cohort identifier column
    df_temp['Cohort'] = cohort_name
    
    dfs.append(df_temp)
    cohort_names.append(cohort_name)
    
    print(f"✅ Loaded {cohort_name} CSV: {len(df_temp)} rows from {csv_path.name}")

# Merge all dataframes
if len(dfs) == 0:
    raise ValueError("No valid cohort CSV files found!")

df = pd.concat(dfs, ignore_index=True)
print(f"\n✅ Merged {len(dfs)} cohort CSV files")
print(f"   Total rows: {len(df)}")
print(f"   Cohorts: {', '.join(cohort_names)}")
print(f"\nColumns: {list(df.columns)}")
print(f"\nFirst few rows:")
print(df.head())
print(f"\nExperiment days: {sorted(df['Experiment_Day'].unique())}")
print(f"\nNumber of animals per cohort:")
print(df.groupby('Cohort')['Animal_ID'].nunique())
print(f"\nTotal unique animals: {df['Animal_ID'].nunique()}")


✅ Loaded 20241125_Cohort1_rotation CSV: 38 rows from cohort_behavioral_analysis.csv
✅ Loaded 20250409_Cohort3_rotation CSV: 30 rows from cohort_behavioral_analysis.csv

✅ Merged 2 cohort CSV files
   Total rows: 68
   Cohorts: 20241125_Cohort1_rotation, 20250409_Cohort3_rotation

Columns: ['Animal_ID', 'Experiment_Day', 'running_threshold_m_per_s', 'running_velocity_avg_m_per_s', 'running_velocity_sd_m_per_s', 'running_time_percentage', 'running_distance_travelled_m', 'running_time_seconds', 'running_total_time_seconds', 'turning_threshold_m_per_s', 'turning_velocity_avg_m_per_s', 'turning_velocity_sd_m_per_s', 'turning_time_percentage', 'turning_distance_turned_m', 'turning_left_percentage', 'turning_right_percentage', 'turning_time_seconds', 'turning_total_time_seconds', 'platform_cross_corr_lag_samples', 'platform_cross_corr_lag_seconds', 'platform_cross_corr_pearson_r', 'platform_cross_corr_p_value', 'platform_gain_encoder_to_turning', 'platform_mean_motor_velocity_m_per_s', 'platf

In [12]:
# Data preparation and conversion
#----------------------------
# Convert running velocity from m/s to cm/s (multiply by 100)
if 'running_velocity_avg_m_per_s' in df.columns:
    df['running_velocity_avg_cm_per_s'] = df['running_velocity_avg_m_per_s'] * 100
else:
    print("⚠️ Warning: 'running_velocity_avg_m_per_s' column not found")

if 'running_velocity_sd_m_per_s' in df.columns:
    df['running_velocity_sd_cm_per_s'] = df['running_velocity_sd_m_per_s'] * 100
else:
    print("⚠️ Warning: 'running_velocity_sd_m_per_s' column not found")

# Note: Turning velocity is already in degrees/s (despite column name suggesting m/s)
# The column name is misleading but the values are in degrees/s
if 'turning_velocity_avg_m_per_s' in df.columns:
    # Keep as is - values are already in degrees/s
    df['turning_velocity_avg_deg_per_s'] = df['turning_velocity_avg_m_per_s']
else:
    print("⚠️ Warning: 'turning_velocity_avg_m_per_s' column not found")

# Normalize cumulative metrics by total recording time
# This accounts for different session durations across experiment days
# Use running_total_time_seconds or turning_total_time_seconds (they should be the same)
if 'running_total_time_seconds' in df.columns:
    total_time_col = 'running_total_time_seconds'
elif 'turning_total_time_seconds' in df.columns:
    total_time_col = 'turning_total_time_seconds'
else:
    total_time_col = None
    print("⚠️ Warning: No total time column found. Cannot normalize distances.")

if total_time_col is not None:
    # Convert seconds to minutes for normalization
    df['total_time_minutes'] = df[total_time_col] / 60.0
    
    # Normalize running distance: m per minute
    if 'running_distance_travelled_m' in df.columns:
        df['running_distance_travelled_m_per_minute'] = df['running_distance_travelled_m'] / df['total_time_minutes']
        print("✅ Created normalized running distance (m/minute)")
    else:
        print("⚠️ Warning: 'running_distance_travelled_m' column not found")
    
    # Normalize turning distance: degrees per minute (note: turning_distance_turned_m is actually in degrees)
    if 'turning_distance_turned_m' in df.columns:
        df['turning_distance_turned_deg_per_minute'] = df['turning_distance_turned_m'] / df['total_time_minutes']
        print("✅ Created normalized turning distance (deg/minute)")
    else:
        print("⚠️ Warning: 'turning_distance_turned_m' column not found")
    
    # Display summary of session durations
    print(f"\n📊 Session duration summary (minutes):")
    print(f"   Mean: {df['total_time_minutes'].mean():.2f} minutes")
    print(f"   Min: {df['total_time_minutes'].min():.2f} minutes")
    print(f"   Max: {df['total_time_minutes'].max():.2f} minutes")
    print(f"\n   By experiment day:")
    for day in sorted(df['Experiment_Day'].unique()):
        day_times = df[df['Experiment_Day'] == day]['total_time_minutes']
        print(f"   {day}: {day_times.mean():.2f} ± {day_times.std():.2f} minutes (n={len(day_times)})")
else:
    # Create dummy normalized columns if normalization not possible
    df['running_distance_travelled_m_per_minute'] = df.get('running_distance_travelled_m', np.nan)
    df['turning_distance_turned_deg_per_minute'] = df.get('turning_distance_turned_m', np.nan)
    print("⚠️ Using non-normalized distances (no time normalization available)")

# Check required columns
required_cols = ['Animal_ID', 'Experiment_Day', 'running_velocity_avg_cm_per_s', 
                 'running_distance_travelled_m_per_minute', 'running_time_percentage',
                 'turning_velocity_avg_deg_per_s', 'turning_distance_turned_deg_per_minute', 
                 'turning_time_percentage']
missing_cols = [col for col in required_cols if col not in df.columns]

if missing_cols:
    print(f"⚠️ Warning: Missing columns: {missing_cols}")
    print(f"Available columns: {list(df.columns)}")
else:
    print("\n✅ All required columns found (including normalized distances)")


✅ Created normalized running distance (m/minute)
✅ Created normalized turning distance (deg/minute)

📊 Session duration summary (minutes):
   Mean: 14.04 minutes
   Min: 0.42 minutes
   Max: 21.14 minutes

   By experiment day:
   Training_day1: 20.64 ± 0.45 minutes (n=4)
   Training_day2: 14.72 ± 4.93 minutes (n=9)
   Training_day3: 10.66 ± 6.61 minutes (n=10)
   Training_day4: 17.30 ± 2.38 minutes (n=9)
   Training_day5: 20.61 ± 0.25 minutes (n=8)
   Training_day6: 20.53 ± 0.24 minutes (n=10)
   Visual_mismatch_day3: 5.95 ± 0.56 minutes (n=9)
   Visual_mismatch_day4: 5.97 ± 0.41 minutes (n=9)

✅ All required columns found (including normalized distances)


In [13]:
# Calculate summary statistics per experiment day
#----------------------------
def calculate_cohort_stats(df: pd.DataFrame, experiment_day: str) -> Dict:
    """Calculate mean and SEM for all mice on a given experiment day."""
    day_data = df[df['Experiment_Day'] == experiment_day].copy()
    
    if len(day_data) == 0:
        return None
    
    n = len(day_data)
    
    # Calculate SEM = SD / sqrt(n)
    def sem(x):
        return x.std() / np.sqrt(n) if n > 1 else 0
    
    stats = {
        'experiment_day': experiment_day,
        'n_mice': n,
        # Running stats
        'running_velocity_avg_cm_per_s_mean': day_data['running_velocity_avg_cm_per_s'].mean(),
        'running_velocity_avg_cm_per_s_sem': sem(day_data['running_velocity_avg_cm_per_s']),
        'running_distance_travelled_m_per_minute_mean': day_data['running_distance_travelled_m_per_minute'].mean(),
        'running_distance_travelled_m_per_minute_sem': sem(day_data['running_distance_travelled_m_per_minute']),
        'running_time_percentage_mean': day_data['running_time_percentage'].mean(),
        'running_time_percentage_sem': sem(day_data['running_time_percentage']),
        # Turning stats
        'turning_velocity_avg_deg_per_s_mean': day_data['turning_velocity_avg_deg_per_s'].mean(),
        'turning_velocity_avg_deg_per_s_sem': sem(day_data['turning_velocity_avg_deg_per_s']),
        'turning_distance_turned_deg_per_minute_mean': day_data['turning_distance_turned_deg_per_minute'].mean(),
        'turning_distance_turned_deg_per_minute_sem': sem(day_data['turning_distance_turned_deg_per_minute']),
        'turning_time_percentage_mean': day_data['turning_time_percentage'].mean(),
        'turning_time_percentage_sem': sem(day_data['turning_time_percentage'])
    }
    
    return stats

# Calculate stats for all experiment days
experiment_days = sorted(df['Experiment_Day'].unique())
cohort_stats = []

for day in experiment_days:
    stats = calculate_cohort_stats(df, day)
    if stats:
        cohort_stats.append(stats)

cohort_stats_df = pd.DataFrame(cohort_stats)
print("\nCohort Summary Statistics (Mean ± SEM) - Normalized by session duration:")
print(cohort_stats_df.to_string(index=False))



Cohort Summary Statistics (Mean ± SEM) - Normalized by session duration:
      experiment_day  n_mice  running_velocity_avg_cm_per_s_mean  running_velocity_avg_cm_per_s_sem  running_distance_travelled_m_per_minute_mean  running_distance_travelled_m_per_minute_sem  running_time_percentage_mean  running_time_percentage_sem  turning_velocity_avg_deg_per_s_mean  turning_velocity_avg_deg_per_s_sem  turning_distance_turned_deg_per_minute_mean  turning_distance_turned_deg_per_minute_sem  turning_time_percentage_mean  turning_time_percentage_sem
       Training_day1       4                            7.510307                           0.439197                                      1.040183                                     0.201948                     20.380672                     3.333131                             6.560279                            0.615400                                   138.808904                                   14.525442                     26.281323              

In [14]:
# Assign consistent colors to each mouse using gnuplot2 palette
#----------------------------
def assign_mouse_colors(df: pd.DataFrame) -> Dict[str, str]:
    """Assign a consistent color to each mouse across all days using gnuplot2 palette."""
    unique_mice = sorted(df['Animal_ID'].unique())
    n_colors = len(unique_mice)
    
    # Use gnuplot2 color palette (similar to gnuplot's default palette)
    # gnuplot2 is a rainbow-like palette going from blue to red
    # Avoid the white end (value 1.0) by using 0 to 0.95 instead of 0 to 1
    # This ensures all colors are visible and distinct
    colors = plt.cm.gnuplot2(np.linspace(0, 0.95, n_colors))
    
    mouse_colors = {mouse: colors[i] for i, mouse in enumerate(unique_mice)}
    return mouse_colors

mouse_colors = assign_mouse_colors(df)
print(f"✅ Assigned colors to {len(mouse_colors)} mice using gnuplot2 palette")
print(f"\nMouse colors:")
for mouse, color in list(mouse_colors.items())[:10]:  # Show first 10
    print(f"  {mouse}: {color}")


✅ Assigned colors to 10 mice using gnuplot2 palette

Mouse colors:
  B6J2717: [0. 0. 0. 1.]
  B6J2718: [0.         0.         0.42352941 1.        ]
  B6J2719: [0.         0.         0.84705882 1.        ]
  B6J2721: [0.21139706 0.         1.         1.        ]
  B6J2722: [0.54227941 0.00705882 0.99294118 1.        ]
  B6J2723: [0.87316176 0.21882353 0.78117647 1.        ]
  B6J2780: [1.         0.43058824 0.56941176 1.        ]
  B6J2781: [1.         0.64235294 0.35764706 1.        ]
  B6J2782: [1.         0.85411765 0.14588235 1.        ]
  B6J2783: [1.         1.         0.41176471 1.        ]


In [18]:
# Create comprehensive plot
#----------------------------
def plot_cohort_behavioral_analysis(df: pd.DataFrame, cohort_stats_df: pd.DataFrame, 
                                     mouse_colors: Dict[str, str], output_path: Path):
    """Create a comprehensive plot showing all running and turning metrics with individual mice and cohort averages."""
    
    experiment_days = sorted(df['Experiment_Day'].unique())
    n_days = len(experiment_days)
    
    # Create figure with 2 rows and 3 columns (6 subplots total)
    # Total height should be 19 cm including x-axis labels
    # Convert cm to inches: 19 cm = 7.48 inches
    fig, axes = plt.subplots(2, 3, figsize=(30, 9))
    
    x_pos = np.arange(n_days)
    
    # Helper function to plot a single metric
    def plot_metric(ax, means, sems, individual_data_dict, title, ylabel):
        """Helper function to plot a metric with individual mice and mean ± SEM."""
        # Plot individual mice
        for mouse, mouse_values in individual_data_dict.items():
            color = mouse_colors[mouse]
            ax.plot(x_pos, mouse_values, 'o-', color=color, alpha=0.7, 
                    linewidth=1.5, markersize=5, label=mouse)
        
        # Plot mean line (black)
        ax.plot(x_pos, means, '-', color='black', linewidth=2.5, zorder=10)
        
        # Plot SEM as opaque grey fill
        ax.fill_between(x_pos, means - sems, means + sems, 
                       color='grey', alpha=0.5, zorder=9)
        
        ax.set_title(title, fontsize=15, fontweight='bold')
        ax.set_ylabel(ylabel, fontsize=15, fontweight='bold')
        ax.set_xticks(x_pos)
        ax.set_xticklabels(experiment_days, rotation=20, ha='right', fontsize=15)
        # Remove grid
        ax.grid(False)
    
    # Collect individual mouse data for each metric
    
    # ========== ROW 1: RUNNING METRICS ==========
    
    # Plot 1: Running Velocity (cm/s)
    ax1 = axes[0, 0]
    running_velocities_dict = {}
    for mouse in df['Animal_ID'].unique():
        mouse_data = df[df['Animal_ID'] == mouse]
        mouse_velocities = []
        for day in experiment_days:
            day_mouse_data = mouse_data[mouse_data['Experiment_Day'] == day]
            if len(day_mouse_data) > 0:
                mouse_velocities.append(day_mouse_data['running_velocity_avg_cm_per_s'].values[0])
            else:
                mouse_velocities.append(np.nan)
        running_velocities_dict[mouse] = mouse_velocities
    
    means = cohort_stats_df['running_velocity_avg_cm_per_s_mean'].values
    sems = cohort_stats_df['running_velocity_avg_cm_per_s_sem'].values
    plot_metric(ax1, means, sems, running_velocities_dict, 
               'Running Velocity per Experiment Day', 'Running Velocity (cm/s)')
    ax1.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=8, ncol=2)
    
    # Plot 2: Total Run Distance (m/minute) - Normalized by session duration
    ax2 = axes[0, 1]
    running_distances_dict = {}
    for mouse in df['Animal_ID'].unique():
        mouse_data = df[df['Animal_ID'] == mouse]
        mouse_distances = []
        for day in experiment_days:
            day_mouse_data = mouse_data[mouse_data['Experiment_Day'] == day]
            if len(day_mouse_data) > 0:
                mouse_distances.append(day_mouse_data['running_distance_travelled_m_per_minute'].values[0])
            else:
                mouse_distances.append(np.nan)
        running_distances_dict[mouse] = mouse_distances
    
    means = cohort_stats_df['running_distance_travelled_m_per_minute_mean'].values
    sems = cohort_stats_df['running_distance_travelled_m_per_minute_sem'].values
    plot_metric(ax2, means, sems, running_distances_dict,
               'Total Run Distance per Experiment Day (Normalized)', 'Run Distance (m/min)')
    
    # Plot 3: Time Spent Running (Percentage)
    ax3 = axes[0, 2]
    running_percentages_dict = {}
    for mouse in df['Animal_ID'].unique():
        mouse_data = df[df['Animal_ID'] == mouse]
        mouse_percentages = []
        for day in experiment_days:
            day_mouse_data = mouse_data[mouse_data['Experiment_Day'] == day]
            if len(day_mouse_data) > 0:
                mouse_percentages.append(day_mouse_data['running_time_percentage'].values[0])
            else:
                mouse_percentages.append(np.nan)
        running_percentages_dict[mouse] = mouse_percentages
    
    means = cohort_stats_df['running_time_percentage_mean'].values
    sems = cohort_stats_df['running_time_percentage_sem'].values
    plot_metric(ax3, means, sems, running_percentages_dict,
               'Time Spent Running per Experiment Day', 'Time Spent Running (%)')
    
    # ========== ROW 2: TURNING METRICS ==========
    
    # Plot 4: Turning Velocity (deg/s)
    ax4 = axes[1, 0]
    turning_velocities_dict = {}
    for mouse in df['Animal_ID'].unique():
        mouse_data = df[df['Animal_ID'] == mouse]
        mouse_velocities = []
        for day in experiment_days:
            day_mouse_data = mouse_data[mouse_data['Experiment_Day'] == day]
            if len(day_mouse_data) > 0:
                mouse_velocities.append(day_mouse_data['turning_velocity_avg_deg_per_s'].values[0])
            else:
                mouse_velocities.append(np.nan)
        turning_velocities_dict[mouse] = mouse_velocities
    
    means = cohort_stats_df['turning_velocity_avg_deg_per_s_mean'].values
    sems = cohort_stats_df['turning_velocity_avg_deg_per_s_sem'].values
    plot_metric(ax4, means, sems, turning_velocities_dict,
               'Turning Velocity per Experiment Day', 'Turning Velocity (deg/s)')
    
    # Plot 5: Total Turn Distance (deg/minute) - Normalized by session duration
    ax5 = axes[1, 1]
    turning_distances_dict = {}
    for mouse in df['Animal_ID'].unique():
        mouse_data = df[df['Animal_ID'] == mouse]
        mouse_distances = []
        for day in experiment_days:
            day_mouse_data = mouse_data[mouse_data['Experiment_Day'] == day]
            if len(day_mouse_data) > 0:
                mouse_distances.append(day_mouse_data['turning_distance_turned_deg_per_minute'].values[0])
            else:
                mouse_distances.append(np.nan)
        turning_distances_dict[mouse] = mouse_distances
    
    means = cohort_stats_df['turning_distance_turned_deg_per_minute_mean'].values
    sems = cohort_stats_df['turning_distance_turned_deg_per_minute_sem'].values
    plot_metric(ax5, means, sems, turning_distances_dict,
               'Total Turn Distance per Experiment Day (Normalized)', 'Turn Distance (deg/min)')
    
    # Plot 6: Time Spent Turning (Percentage)
    ax6 = axes[1, 2]
    turning_percentages_dict = {}
    for mouse in df['Animal_ID'].unique():
        mouse_data = df[df['Animal_ID'] == mouse]
        mouse_percentages = []
        for day in experiment_days:
            day_mouse_data = mouse_data[mouse_data['Experiment_Day'] == day]
            if len(day_mouse_data) > 0:
                mouse_percentages.append(day_mouse_data['turning_time_percentage'].values[0])
            else:
                mouse_percentages.append(np.nan)
        turning_percentages_dict[mouse] = mouse_percentages
    
    means = cohort_stats_df['turning_time_percentage_mean'].values
    sems = cohort_stats_df['turning_time_percentage_sem'].values
    plot_metric(ax6, means, sems, turning_percentages_dict,
               'Time Spent Turning per Experiment Day', 'Time Spent Turning (%)')
    
    # Only add xlabel to bottom row plots
    for ax in [ax4, ax5, ax6]:
        ax.set_xlabel('')  # Remove xlabel as requested
    
    plt.tight_layout()
    plt.savefig(output_path, format='svg', bbox_inches='tight')
    print(f"✅ Saved plot to: {output_path}")
    plt.close(fig)

# Create the plot
# Use simplified filename based on cohort directory names
cohort_str = "_".join(cohort_names)
plot_path = output_dir / f"{cohort_str}_averages.svg"
plot_cohort_behavioral_analysis(df, cohort_stats_df, mouse_colors, plot_path)


✅ Saved plot to: /home/ikharitonov/RANCZLAB-NAS/data/ONIX/20241125_Cohort1_rotation/20241125_Cohort1_rotation_20250409_Cohort3_rotation_averages.svg


In [ ]:
# Save summary statistics to CSV
#----------------------------
# Use simplified filename based on cohort directory names
cohort_str = "_".join(cohort_names)
summary_csv_path = output_dir / f"{cohort_str}_averages.csv"
cohort_stats_df.to_csv(summary_csv_path, index=False)
print(f"✅ Saved summary statistics to: {summary_csv_path}")

# Display the summary
print("\nSummary Statistics (Mean ± SEM) - Combined across all cohorts:")
print(cohort_stats_df.to_string(index=False))

# Also show breakdown by cohort if multiple cohorts
if len(cohort_names) > 1:
    print(f"\n\nBreakdown by Cohort and Experiment Day:")
    for cohort in cohort_names:
        cohort_data = df[df['Cohort'] == cohort]
        print(f"\n{cohort}:")
        print(f"  Number of animals: {cohort_data['Animal_ID'].nunique()}")
        print(f"  Experiment days: {sorted(cohort_data['Experiment_Day'].unique())}")


In [ ]:
# Optional: Create individual plots for each metric (larger, more detailed)
#----------------------------
def plot_individual_metric(df: pd.DataFrame, cohort_stats_df: pd.DataFrame,
                          mouse_colors: Dict[str, str], metric: str, 
                          metric_label: str, ylabel: str, output_path: Path):
    """Create a detailed plot for a single metric."""
    
    experiment_days = sorted(df['Experiment_Day'].unique())
    n_days = len(experiment_days)
    x_pos = np.arange(n_days)
    
    fig, ax = plt.subplots(figsize=(12, 8))
    
    # Plot individual mice
    for mouse in df['Animal_ID'].unique():
        mouse_data = df[df['Animal_ID'] == mouse]
        mouse_values = []
        
        for day in experiment_days:
            day_mouse_data = mouse_data[mouse_data['Experiment_Day'] == day]
            if len(day_mouse_data) > 0:
                mouse_values.append(day_mouse_data[metric].values[0])
            else:
                mouse_values.append(np.nan)
        
        color = mouse_colors[mouse]
        ax.plot(x_pos, mouse_values, 'o-', color=color, alpha=0.7, 
                linewidth=2, markersize=8, label=mouse)
    
    # Plot cohort average ± SEM
    mean_col = f"{metric}_mean"
    sem_col = f"{metric}_sem"
    
    if mean_col in cohort_stats_df.columns and sem_col in cohort_stats_df.columns:
        means = cohort_stats_df[mean_col].values
        sems = cohort_stats_df[sem_col].values
        
        ax.errorbar(x_pos, means, yerr=sems, fmt='o-', color='black', 
                   linewidth=4, markersize=12, capsize=8, capthick=3, 
                   label='Cohort Mean ± SEM', zorder=10)
    
    ax.set_xlabel('Experiment Day', fontsize=14, fontweight='bold')
    ax.set_ylabel(ylabel, fontsize=14, fontweight='bold')
    ax.set_title(f'{metric_label} per Experiment Day', fontsize=16, fontweight='bold')
    ax.set_xticks(x_pos)
    ax.set_xticklabels(experiment_days, rotation=45, ha='right', fontsize=12)
    ax.grid(True, alpha=0.3)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=10, ncol=2)
    
    plt.tight_layout()
    plt.savefig(output_path, format='svg', bbox_inches='tight')
    print(f"✅ Saved plot to: {output_path}")
    plt.close(fig)

# Create individual plots (optional - uncomment if needed)
# plot_individual_metric(df, cohort_stats_df, mouse_colors, 
#                       'running_velocity_avg_cm_per_s', 'Running Velocity', 
#                       'Running Velocity (cm/s)', 
#                       output_dir / "running_velocity_detailed.svg")
# 
# plot_individual_metric(df, cohort_stats_df, mouse_colors,
#                       'running_distance_travelled_m_per_minute', 'Total Run Distance (Normalized)',
#                       'Run Distance (m/min)',
#                       output_dir / "running_distance_detailed.svg")
# 
# plot_individual_metric(df, cohort_stats_df, mouse_colors,
#                       'running_time_percentage', 'Time Spent Running',
#                       'Time Spent Running (%)',
#                       output_dir / "running_time_percentage_detailed.svg")
# 
# plot_individual_metric(df, cohort_stats_df, mouse_colors,
#                       'turning_velocity_avg_deg_per_s', 'Turning Velocity',
#                       'Turning Velocity (deg/s)',
#                       output_dir / "turning_velocity_detailed.svg")
# 
# plot_individual_metric(df, cohort_stats_df, mouse_colors,
#                       'turning_distance_turned_deg_per_minute', 'Total Turn Distance (Normalized)',
#                       'Turn Distance (deg/min)',
#                       output_dir / "turning_distance_detailed.svg")
# 
# plot_individual_metric(df, cohort_stats_df, mouse_colors,
#                       'turning_time_percentage', 'Time Spent Turning',
#                       'Time Spent Turning (%)',
#                       output_dir / "turning_time_percentage_detailed.svg")
